# Task instructions

Manually extract key financial data for the last three fiscal years from the 10-K filings of Microsoft, Tesla, and Apple. Following the data collection, use Python to analyze this data, focusing on trends and insights that could inform the development of an AI-powered financial chatbot.

### Step 1: Data extraction

- Navigate to the SEC's EDGAR database:

    - Microsoft
    - Tesla
    - Apple

- Manual extraction:

    - For each company, find the 10-K filings for the last three fiscal years.
    - Extract the following financial figures: Total Revenue, Net Income, Total Assets, Total Liabilities, and Cash Flow from Operating Activities.

- Organize Your Data:

    - Compile the extracted data into an Excel spreadsheet for easy reference during your Python analysis.